In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/face-data/static/faces/aman_123/aman_33.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_5.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_44.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_41.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_1.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_29.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_25.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_12.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_32.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_49.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_6.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_0.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_31.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_28.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_45.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_34.jpg
/kaggle/input/face-data/static/faces/aman_123/aman_48.jpg
/kaggle/input/face

In [71]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [72]:
# model = Sequential([
#     Dense(units=16, input_shape=(1,), activation='relu'),
#     Dense(units=32, activation='relu'),
#     Dense(units=2, activation='softmax')
# ])

In [73]:
# model.summary()

In [74]:
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics='accuracy')

In [76]:
# import os
from PIL import Image
# import numpy as np

# Define the directory path
directory = "/kaggle/input/face-data/static/faces/aman_123/"

# Initialize an empty list to store the images
images_aman = []

# List all files in the directory
file_list = os.listdir(directory)

# Iterate through the files and load them as images
for filename in file_list:
    # Check if the file is an image (you can further filter by file extension)
    if filename.endswith(".jpg"):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Open and preprocess the image
        img = Image.open(file_path)
        img = img.resize((224, 224))  # Resize the image to your desired dimensions
        img = np.array(img)  # Convert to a NumPy array
        images_aman.append(img)

# Convert the list of images into a NumPy array
images_array_aman = np.array(images_aman)

# Add a batch dimension to the images
images_array_aman = np.expand_dims(images_array_aman, axis=0)

# Check the shape of the array (should be num_images x height x width x channels)
print("Shape of the images array:", images_array_aman.shape)
images_array_aman[0][0].shape

Shape of the images array: (1, 50, 224, 224, 3)


(224, 224, 3)

In [77]:
# import os
# from PIL import Image
# import numpy as np

# Define the directory path
directory = "/kaggle/input/face-data/static/faces/sai_123/"

# Initialize an empty list to store the images
images_sai = []

# List all files in the directory
file_list = os.listdir(directory)

# Iterate through the files and load them as images
for filename in file_list:
    # Check if the file is an image (you can further filter by file extension)
    if filename.endswith(".jpg"):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Open and preprocess the image
        img = Image.open(file_path)
        img = img.resize((224, 224))  # Resize the image to your desired dimensions
        img = np.array(img)  # Convert to a NumPy array
        images_sai.append(img)

# Convert the list of images into a NumPy array
images_array_sai = np.array(images_sai)

# Add a batch dimension to the images
images_array_sai = np.expand_dims(images_array_sai, axis=0)

# Check the shape of the array (should be num_images x height x width x channels)
print("Shape of the images array:", images_array_sai.shape)
images_array_sai[0][0].shape

Shape of the images array: (1, 50, 224, 224, 3)


(224, 224, 3)

In [78]:
mobile = tf.keras.applications.mobilenet.MobileNet(weights="imagenet",include_top=False)
# mobile.summary()

In [79]:
x = GlobalAveragePooling2D()(mobile.output)
linear_vector = Dense(10, activation='linear')(x)
model = Model(inputs=mobile.input, outputs=linear_vector)

feature_ex = mobile.layers[-5].output
model = Model(inputs=mobile.input, outputs = feature_ex)

for layer in model.layers[:-23]:
    layer.trainable = False

In [89]:
linear_vectors1 = model.predict(images_array_aman[0])
linear_vectors3 = model.predict(images_array_aman[0])
linear_vectors2 = model.predict(images_array_sai[0])
# linear_vectors.get_weights()
# print(linear_vectors)

2/2 [==============================] - 1s 283ms/step


In [93]:
# import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming linear_vectors1 and linear_vectors2 are two sets of linear vectors
# Each set has shape (batch_size, vector_length)

# Flatten the arrays to remove the batch dimension
flat_vectors1 = linear_vectors1.reshape(linear_vectors1.shape[0], -1)
flat_vectors2 = linear_vectors2.reshape(linear_vectors2.shape[0], -1)

# Normalize the vectors
norm_vectors1 = flat_vectors1 / np.linalg.norm(flat_vectors1, axis=1)[:, np.newaxis]
norm_vectors2 = flat_vectors2 / np.linalg.norm(flat_vectors2, axis=1)[:, np.newaxis]

# Compute cosine similarity between the two sets of vectors
similarity = cosine_similarity(norm_vectors1, norm_vectors2)

# Define a threshold to determine if the vectors are matching
threshold = 0.99  # Adjust the threshold as needed

# Check if any pair of vectors has a similarity greater than the threshold
match = (similarity >= threshold).any()

if match:
    print("The feature vectors are from similar images.")
else:
    print("The feature vectors are not from similar images.")


The feature vectors are not from similar images.


In [24]:
model.fit(x=train_data, epochs=10, verbose=0)

ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1052, in train_step
        self._validate_target_and_loss(y, loss)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1016, in _validate_target_and_loss
        raise ValueError(

    ValueError: No loss found. You may have forgotten to provide a `loss` argument in the `compile()` method.
